In [7]:
#Kiwi Randolph
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np 

In [61]:
trainSIFT = np.load('train_D-1.npy', allow_pickle=True)
testSIFT = np.load('test_D-1.npy', allow_pickle=True)
print(trainSIFT.shape)

(1888,)


In [101]:
sift_train_concatenated = np.concatenate(trainSIFT, axis=0)
sift_test_concatenated = np.concatenate(testSIFT, axis=0)
clusterTrain, labelTrain = kmeans_clustering(50, sift_train_concatenated)
clusterTest, labelTest = kmeans_clustering(50, sift_test_concatenated)

In [98]:
def kmeans_clustering(cluster, trainf):
    kmeans = KMeans(n_clusters = cluster, random_state = 0, init = 'k-means++', n_init = 15, max_iter=100)

    kvalue = kmeans.fit(trainf)
    labels = kmeans.predict(trainf)
    
    cluster = kmeans.cluster_centers_
    
    return cluster, labels

def build_bow_representation(sift_features, labels, cluster_centers):
    bow_representation = []
    for i, features in enumerate(sift_features):
        bow_vector = np.bincount(labels[i], minlength=len(cluster_centers))
        bow_representation.append(bow_vector)
    return np.array(bow_representation)


def kNN(train_bow_list, trainingLabels, test_bow_list, n_neighbors):
    knn_model = KNeighborsClassifier(n_neighbors=n_neighbors)
    
    knn_model.fit(train_bow_list, trainingLabels)
    
    predictions = knn_model.predict(test_bow_list)
    
    return predictions

In [102]:
trainLabel = np.load('train_gs-1.npy')
testLabel = np.load('test_gs.npy')
trainLabel = np.load('train_gs-1.npy').reshape(-1)
testLabel = np.load('test_gs.npy').reshape(-1)
print(trainLabel.shape)
print(testLabel.shape)
labelTrain_reshaped = np.array_split(labelTrain, len(trainSIFT))
labelTest_reshaped = np.array_split(labelTest, len(testSIFT))

BOWtrain = build_bow_representation(trainSIFT, labelTrain_reshaped, clusterTrain)
BOWtest = build_bow_representation(testSIFT, labelTest_reshaped, clusterTest)


# BOWtrain = assign_bow(trainSIFT, clusterTrain, trainLabel)
# BOWtest = assign_bow(testSIFT, clusterTest, testLabel)
# print(BOWtrain)
# print(BOWtest)


(1888,)
(800,)


In [106]:
predictions = kNN(BOWtrain, trainLabel, BOWtest, n_neighbors=20)


# # Flatten the arrays for compatibility
# testLabel_flat = testLabel.flatten()
# predicted_labels_flat = predicted_labels.flatten()



accuracy = accuracy_score(testLabel, predictions)
print("Accuracy:", (accuracy * 100), "%")
conf_matrix = confusion_matrix(testLabel, predictions)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 12.5 %
Confusion Matrix:
[[12  6  6 11 13 23 16 13]
 [10 10  7  7 13 22 17 14]
 [11  7  3  8 17 22 16 16]
 [13  9  3  9 18 24 14 10]
 [13 10  6 10 15 19 19  8]
 [12  7  4 12 10 22 13 20]
 [ 6  7  7  6 11 30 11 22]
 [17  9  7  3  9 18 19 18]]
